<a href="https://colab.research.google.com/github/kkipngenokoech/Material-Science/blob/main/fall/energy_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install atomic simulation environment and simulation code

This sets up our simulation environment. ASE (Atomic Simulation Environment) builds molecules, and GPAW (a DFT code) simulates molecules and computes energies using electronic structure calculations and by approximating the electron cloud.

In [ ]:
# install ASE (atomic simulation environment) using Linux
!apt install ase

# install GPAW (the simulation code) using Linux
!apt install python3-mpi4py cython3 libxc-dev gpaw-data
!pip -q install gpaw

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  blt fonts-dejavu-core fonts-font-awesome fonts-glyphicons-halflings
  fonts-lyx libimagequant0 libjs-bootstrap liblbfgsb0 libraqm0 libxsimd-dev
  python-matplotlib-data python3-appdirs python3-ase python3-attr
  python3-beniget python3-brotli python3-bs4 python3-chardet python3-cycler
  python3-dateutil python3-decorator python3-fonttools python3-fs python3-gast
  python3-html5lib python3-kiwisolver python3-lxml python3-lz4
  python3-matplotlib python3-mpmath python3-numpy python3-olefile
  python3-packaging python3-pil python3-pil.imagetk python3-ply
  python3-pythran python3-scipy python3-soupsieve python3-spglib python3-sympy
  python3-tk python3-tz python3-ufolib2 python3-unicodedata2
  python3-webencodings tk8.6-blt2.5 unicode-data
Suggested packages:
  python-ase-doc blt-demo libxsimd-doc python3-netcdf4 python3-pycodcif
  python-

In [ ]:
# import tools from ASE
from ase import Atoms, Atom
from ase.build import molecule as ase_molecule
from ase.data.pubchem import pubchem_atoms_search
from ase.visualize import view

# import tools from GPAW
from gpaw import GPAW, PW

# import tools for plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# import tools for numerical calculations
import numpy as np

# 1. Create the molecule &#129337;

Change the CID to simulate different molecules—e.g., 297 for methane. This pulls atomic positions from PubChem.

In [ ]:
"""
define molecule using the PubChem database of the National Library of Medecine

methane         = 297
propane         = 6334
ethanol         = 702
octane          = 10907
oxygen          = 977
carbon dioxide  = 280
water           = 962
hydrogen        = n/a   (type the command "molecule = ase_molecule('H2')"
                        instead of "molecule = pubchem_atoms_search(cid=XXX)")

"""
molecule = pubchem_atoms_search(cid=280)
molecule.set_cell([7, 7, 7], scale_atoms=False)
molecule.center()

# 2. Visualize the molecule &#x1F50D;

In [ ]:
# show molecule
view(molecule, viewer='x3d', block=True)

# 3. Calculate the energy of the molecule &#x1F4BB;

Run the simulation. This solves the Kohn-Sham equations to get the total energy for each molecule.

In [ ]:
# set simulation parameters
calc = GPAW(xc='PBE',
            kpts=(1,1,1),
            mode=PW(700),
            basis='dzp')
molecule.set_calculator(calc)

# run computer simulation (using quantum mechanics to compute the electron cloud)
energy = molecule.get_total_energy()

print("*****")
print(" The energy of the molecule is "+str(energy)+" eV")
print("*****")

# 4. Visualize the electron cloud of the molecule &#x269B;

The electron cloud represents the probability distribution of an electron's position around an atom or molecule, derived from simulations. It shows where the electron is most likely to be found, with denser areas indicating higher probability. This is the 'cloud' that determines bonding.

In [ ]:
# convert electron cloud
n = calc.get_all_electron_density(gridrefinement=4)
nred = n[::3,::3,::3]
X, Y, Z = np.mgrid[0:7:60j, 0:7:60j, 0:7:60j]

# show electron cloud
fig = go.Figure(data=go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=nred.flatten(),
    isomin=0.1,
    isomax=1,
    surface_count=5,
    colorbar_nticks=5,
    caps=dict(x_show=False, y_show=False),
    opacity=0.3
    ))

fig.show()